In [1]:
from pydrake.all import (MathematicalProgram, Solve, MonomialBasis,
                         DiagramBuilder, Evaluate, LogVectorOutput, Simulator,
                         SymbolicVectorSystem, Variable, ToLatex, Polynomial,
                         VectorSystem, eq, ge, le, Formula, Expression, Evaluate,
                         LeafSystem, AbstractValue,
                         )

from pydrake.all import (PiecewisePolynomial, ModelInstanceIndex,
    RotationMatrix, RigidTransform, Rgba, Box, Sphere, BaseField,
    Evaluate, Fields, PointCloud, MeshcatAnimation)

from IPython.display import clear_output

import os
import copy
import time
import zarr
from datetime import datetime
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from dataclasses import dataclass
from dccm_quasistatic.utils.math_utils import (create_square_symmetric_matrix_from_lower_tri_array,
                                               get_n_lower_tri_from_matrix_dim,
                                               matrix_inverse)
from dccm_quasistatic.utils.sample_generator import (SampleGenerator, SampleGeneratorParams)
from dccm_quasistatic.controller.dccm_params import DCCMParams
from dccm_quasistatic.controller_synthesizer.dccm_synthesizer import DCCMSynthesizer
 
from qsim.parser import (
    QuasistaticParser,
    QuasistaticSystemBackend,
    GradientMode,
)

from qsim.simulator import ForwardDynamicsMode, InternalVisualizationType
from qsim.model_paths import models_dir, package_paths_dict

/home/shaoyuan/Documents/Software/quasistatic_simulator/robotics_utilities/qp_derivatives/qp_derivatives.py:2: DrakeDeprecationWarning: Please import from the pydrake.solvers module directly, instead of the deprecated pydrake.solvers.mathematicalprogram submodule. The deprecated code will be removed from Drake on or after 2023-05-01.
  import pydrake.solvers.mathematicalprogram as mp


In [2]:
def synthesize_controller(sample_generator_params: SampleGeneratorParams, dccm_params: DCCMParams):
    file_prefix = "saved_dccm_coeffs/circ_"
    base_filename = f"samples{sample_generator_params.n_samples}_lbw{sample_generator_params.log_barrier_weight}_deg{dccm_params.deg}_beta{str(dccm_params.beta).split('.')[1]}.npy"
    
    # Set up dynamical system
    package_paths_dict["dccm_quasistatic"] =  "/home/shaoyuan/Documents/Software/dccm_quasistatic"
    q_model_path = os.path.join("../../models", "q_sys", "box_pushing.yml")
    q_parser = QuasistaticParser(q_model_path)
    q_sim = q_parser.make_simulator_cpp()
    q_sim_py = q_parser.make_simulator_py(InternalVisualizationType.Cpp)
    
    sample_generator = SampleGenerator(sample_generator_params, q_sim=q_sim, q_sim_py=q_sim_py, parser=q_parser)

    init_wijc = None
    init_lijc = None

    if sample_generator_params.log_barrier_weight == 10:
        samples = sample_generator.generate_circular_traj_samples(b_r = 1.5, s_r_buff = 1.1, visualize=False)
        # init_base_filename = f"samples{sample_generator_params.n_samples}_lbw{sample_generator_params.log_barrier_weight}_deg{dccm_params.deg}_beta{1}.npy"
        # init_wijc = np.load(f"{file_prefix}wijc_{init_base_filename}")
        # init_lijc = np.load(f"{file_prefix}lijc_{init_base_filename}")

    elif sample_generator_params.log_barrier_weight == 100:
        samples = sample_generator.generate_circular_traj_samples(b_r = 1.1, s_r_buff = 0.22, visualize=False)

    dccm_synth = DCCMSynthesizer(dccm_params)
    success, wijc, lijc = dccm_synth.calculate_dccm_from_samples(*samples, init_wijc, init_lijc)
    if success:
        np.save(f"{file_prefix}wijc_{base_filename}", wijc)
        np.save(f"{file_prefix}lijc_{base_filename}", lijc)
    return success, dccm_synth.solution_status



In [3]:
# create my missing lbw 10, 100 sample, beta = 0.1
# sample_generator_params = SampleGeneratorParams(
#     log_barrier_weight=10,
#     n_samples=100,
#     actuated_collision_geomtery_names=["hand::collision"]
# )

# dccm_params = DCCMParams(
#     dim_x=5,
#     dim_u=2,
#     deg=4,
#     beta=0.1,
#     n_geodesic_segments=1,
# )
# success = synthesize_controller(sample_generator_params, dccm_params)

In [4]:
# sample_generator_params = SampleGeneratorParams(
#     log_barrier_weight=10,
#     n_samples=500,
#     actuated_collision_geomtery_names=["hand::collision"]
# )

# dccm_params = DCCMParams(
#     dim_x=5,
#     dim_u=2,
#     deg=4,
#     beta=0.1,
#     n_geodesic_segments=1,
# )

# betas = [0.5]
# success_log = []
# for beta in betas:
#     dccm_params.beta = beta
#     success = synthesize_controller(sample_generator_params, dccm_params)
#     success_log.append(success)
#     if success:
#         print(f"Successfully synthesized controller with beta = {beta}")
#     else:
#         print(f"Failed to synthesize controller with beta = {beta}")

# print(f"success log: {success}")



In [5]:
sample_generator_params = SampleGeneratorParams(
    log_barrier_weight=10,
    n_samples=10,
    actuated_collision_geomtery_names=["hand::collision"]
)

dccm_params = DCCMParams(
    dim_x=5,
    dim_u=2,
    deg=4,
    beta=0.1,
    n_geodesic_segments=1,
)

# betas = [0.5, 0.9]
# for beta in betas:
#     dccm_params.beta = beta
#     success = synthesize_controller(sample_generator_params, dccm_params)
#     if success:
#         print(f"Successfully synthesized controller with beta = {beta}")
#     else:
#         print(f"Failed to synthesize controller with beta = {beta}")

sample_generator_params.log_barrier_weight = 10
dccm_params.beta = 0.1

deg_list = [6]
n_samples_list = [100, 500]
date_time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
results_file = f"synthesis_results/synth_results_{date_time_str}.zarr"
result_group = zarr.open_group(results_file, mode='a')
params_log = []
success_log = []
solution_status_log = []

for deg in deg_list:
    dccm_params.deg = deg
    for n_samples in n_samples_list:
        
        sample_generator_params.n_samples = n_samples
        success, soln_status = synthesize_controller(sample_generator_params, dccm_params)
        params_log.append(f"samples{sample_generator_params.n_samples}_lbw{sample_generator_params.log_barrier_weight}_deg{dccm_params.deg}_beta{str(dccm_params.beta).split('.')[1]}")
        success_log.append(success)
        solution_status_log.append(soln_status)

        result_group["params"] = params_log
        result_group["success"] = success_log
        result_group["solution_status"] = solution_status_log
        
        if success:
            print(f"Successfully synthesized controller with n_samples = {n_samples}")
        else:
            print(f"Failed to synthesize controller with n_samples = {n_samples}")
            break





Start solving DCCM
Solver succeeded:  True  in  12457.182643413544  seconds
num_infeasible_constraints: 0
solver is:  Mosek
solution status 1
rescode 0
wijc:
 [[ 4.90492839e-02  1.73751979e-01 -4.14766737e-02 ...  2.10905556e+01
  -1.03713238e+01  4.94014120e+00]
 [-7.94114599e-03 -4.79229530e-03 -6.64886661e-02 ...  2.86516011e+00
  -8.41927406e+00  2.02264299e+00]
 [ 2.78123458e-02  1.48380846e-01 -2.14155630e-02 ...  2.43584299e+01
  -4.16002365e+00  3.91246122e+00]
 ...
 [ 1.66567259e-02  1.05880776e-01  9.94083249e-02 ... -9.70558082e+00
  -1.24823686e+01 -1.05695223e+00]
 [-1.28715059e-02 -3.69306374e-02 -2.39746231e-02 ... -2.14026477e-01
  -3.67086305e+00 -1.50196794e+00]
 [-8.68072832e-03 -3.45492624e-02  2.08648680e-03 ...  2.51620759e+00
  -3.42485697e+00 -1.13190520e+00]]

lijc:
 [[ 0.00569435 -0.00959518  0.06398999 ... -1.46028274 -0.00998081
  -0.45373577]
 [ 0.01510342 -0.01947781 -0.01223172 ...  0.06391641  0.79863834
   0.589628  ]
 [ 0.0614165  -0.01302748  0.216928

In [6]:
import zarr
import numpy as np
# results_file = "synthesis_results/synth_results_2023-05-11_22-19-51.zarr"
res = zarr.open_group(results_file, mode='r')
print(np.array([i for i in zip(res["params"][:], res["success"][:], res["solution_status"][:])]))


[['samples100_lbw10_deg6_beta1' 'True' '1']
 ['samples500_lbw10_deg6_beta1' 'True' '1']]
